## Calculating the date first observed and frequency of irrigated cropping

Script is divided into two sections:

1. The first part creates a large rasterstack of all the irrigated area files and exports it as netcdf.
2. The rasterstack is then loaded as an xarray object and per-pixel frequency and date first observed is calculated.

In [4]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import fiona
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal, ogr
sys.path.append('src/')
import SpatialTools

### User Inputs

In [8]:
directory = "results/NE/"
suffix = "_Irrigated_LS_masked"
prefix = "Tasmania_Summer"
results = "results/frequency_etal_NE/"

### Part 1: Create rasterstack of irrigation 


In [23]:
#list of years to help for-loop iterate through folders
x = range(2000,2019,1)
years = []
for i in x:
    nextyear = str(i + 1)[2:]
    y = str(i) + "_" + nextyear
    years.append(str(y))
# removing years that didn't work
years =  [e for e in years if e not in ('2011_12', '2012_13')]
years.sort()

#list of folders to help with loop
folders = os.listdir(directory)
folders.sort()

# Convert our shapefiles into tiffs, add them to an xarray and 
# then export as netcdf
def convertIrrShpToTiff(shp, year):  
    #open a tif and get transform info
    tif = shp[:-23]+"multithreshold_65Thres"+suffix[10:]+".tif"
    ds = xr.open_rasterio(tif).squeeze()
    transform, proj = SpatialTools.geotransform(ds, (ds.x, ds.y), epsg=3577)
    rows,cols = ds.shape
    #turn vector in numpy array
    shp_arr = SpatialTools.rasterize_vector(shp, cols=cols, rows=rows, geo_transform=transform, projection=proj)
    #convert numpy array inot xarray
    shp_xr = xr.DataArray(shp_arr, coords = [ds.y, ds.x], dims = ['y', 'x'])
    #append xarray to list
    da_list.append(shp_xr)

da_list = []
for year, folder in zip(years, folders): 
    print("\r", "working on year: " + year, end = '')
    convertIrrShpToTiff(directory+folder+"/"+prefix+ year + suffix+".shp", year)

#generate date ranges to use as cooridnates in xrray dataset
dates = pd.date_range(start='1/1/2000', end='1/01/2019', freq='Y')
dates = dates.drop([pd.Timestamp('2011-12-31'), pd.Timestamp('2012-12-31')])
#concatenate all xarrays in list to a single multi-dim xarray with time ('dates') as coords.
da = xr.concat(da_list, dim=dates).rename({'concat_dim':'time'}).rename('Irrigated_Area')
#convert to dataset
ds = da.to_dataset()
#export netcdf
ds.to_netcdf(results + "annual_irrigation.nc")

 working on year: 2018_19

### Part 2: Calculate per pixel frequency et al.



In [29]:
#create parallized functions for calculting sum and nanargmax
def count_irrigation(x, dim):
    return xr.apply_ufunc(np.sum, x, dask='parallelized',
                          input_core_dims=[[dim]],
                          kwargs={'axis': -1})

def IrrigationFirstOccurs(x, dim):
    """
    Calculating the time (indice) at which the first occurence of 
    Irrigation occurs (per-pixel). This works because np.nanargmax:
    "In cases of multiple occurrences of the maximum values,
    the indices corresponding to the first occurrence are returned."
    """
    return xr.apply_ufunc(np.nanargmax, x, dask='parallelized',
                          input_core_dims=[[dim]],
                          kwargs={'axis': -1})


In [46]:
count = count_irrigation(ds.Irrigated_Area, dim='time')
frequency = count / len(ds.time)
firstOccured = IrrigationFirstOccurs(ds.Irrigated_Area, dim='time')
yearsIrrigated = len(ds.time)-firstOccured 
normalisedFrequency = count / yearsIrrigated

#convert first observed to an array with the date (year)
dates = [t for t in range(2000,2019,1)]
dates =  [e for e in dates if e not in (2011, 2012)]
dates = np.asarray(dates)

def timey(ind, time):
    func = time[ind]
    return func

firstOccuredDates = timey(firstOccured, dates)

In [49]:
#mask out areas that return non-sensical values using the normalised frequency xarray
firstOccuredDates = np.where(normalisedFrequency.values > 0, firstOccuredDates, np.nan)
yearsIrrigated = yearsIrrigated.where(normalisedFrequency.values >0)

In [52]:
#export geotiffs
transform, projection = SpatialTools.geotransform(ds, (ds.x, ds.y), epsg=3577)

SpatialTools.array_to_geotiff(results+'firstOccuredDates_alltime.tif',
              firstOccuredDates, geo_transform = transform, 
              projection = projection, 
              nodata_val=0)


In [ ]:
# SpatialTools.array_to_geotiff(results+'rawFrequency_alltime.tif',
#               frequency.values, geo_transform = transform, 
#               projection = projection, 
#               nodata_val=0)

# SpatialTools.array_to_geotiff(results+'count_alltime.tif',
#               count.values, geo_transform = transform, 
#               projection = projection, 
#               nodata_val=0)

# SpatialTools.array_to_geotiff(results+'firstOccured_alltime.tif',
#               firstOccured.values, geo_transform = transform, 
#               projection = projection, 
#               nodata_val=0)

# SpatialTools.array_to_geotiff(results+'yearsIrrigated_alltime.tif',
#               yearsIrrigated.values, geo_transform = transform, 
#               projection = projection, 
#               nodata_val=0)

# SpatialTools.array_to_geotiff(results+'normalisedFrequency_alltime.tif',
#               normalisedFrequency.values, geo_transform = transform, 
#               projection = projection, 
#               nodata_val=0)